#### These handcodes are written by me throughout my ML assignments, they are hand written functions in place of built in functions. Please do not submit these as your own. These are for educational reference only, thanks!

In [46]:
import numpy as np

### Finding eigenvector by gradient ascent - SVD

In [47]:
# x is the data matrix you are trying to find the eigenvectors on
# w is your eigenvectors


norm = lambda x: sum(x**2)**0.5     #normalization function


def eigenvector(x,learn_rate):
    w = np.random.normal(0,1,(2,1))     #initialize eigenvectors with random numbers
    w = w / norm(w)                     #normalize the vectors, L2 norm
    gradient = 2*x.dot(x.T).dot(w)      #for non-square SVD, we square it by multiplying data.dot(data.T) 
    w_new = w+learn_rate*gradient
    counter = 1
    while norm(w_new-w) > 0.001:
        w = w_new
        gradient = 2*x.dot(x.T).dot(w)
        w_new = w+learn_rate*gradient
        w_new = w_new / norm(w_new)
        counter += 1
    return w_new



### Partial discreet fourier transform + window

In [48]:
# sample rate is your time domain data matrix

def dft(sample_rate): 
    N=1600
    F= np.zeros((N,N),dtype=complex)
    y= 0.0
    f= sample_rate/N
    for f in range(N): #iterate through N=1600
        for n in range(N): #double iterate to move n, while f keeps track
            y = np.exp(-2J*np.pi*f*(n/N))
            F[f][n] = y #now have row index[f] and column index[n] to append y to
    return F

In [49]:
# this is the second part of dft, with the hann window
# data is your time domain data matrix

def window(data):
    N=1600
    hann = np.hanning(N)
    x_matrix= np.zeros((N,78))
    x1 = data[0:N]
    x_matrix[:,0] = x1*hann
    for i in range(1,78): #i is index for row of matrix, row matrix=78
        x2 = data[N//2:1600+N//2]*hann
        #print(N//2,' ',1600+N//2)
        x_matrix[:,i] = x2 #appends x2 coloumn-wise
        N = N + 1600 #moves N so N not stagnant, since for function only moves i
    return x_matrix


### Inverse DFT

In [50]:
# data is your frequency domain data matrix

def reverse_window(data):
    N=1600
    x_list= []
    first = data[0][:800]
    x_list.extend(first)
    for i in range(77):   #length of original window
        x = data[i][N//2:N] + data[i+1][0:N//2]  #reverse hann window
        x_list.extend(x)
        
    x_list.extend(data[77][:-800])
    return x_list

### K means - 1D

In [51]:
# data is for single dimension 1d array data
# k is number of clusters

def k_means(data,k):
    random_kmeans = np.random.choice(data,k) #generate dummy kmeans 
    counter = 1
    points_list = np.zeros(len(data)) #empty list to hold distance points from k_means, for every data point
    while counter < 100:  #define max iterations
            for i in range(len(data)):  #iterate through each datapoint    
                new_distance = np.sqrt((data[i]-random_kmeans)**2) #l2 norm distance for each data point
                #print(new_distance)
                points_list[i] = (np.argmin(new_distance)) #for each point, return index of min, result in (0,1)
                #print(len(points_list))
            mean1 = data[points_list == 0].mean() #get all indices with 0, mean
            mean2 = data[points_list == 1].mean() #get all indices with 1, mean
            random_kmeans = np.array([mean1, mean2]) #update dummy mean
            counter = counter+1
            k_len = len(data[points_list == 0]),len(data[points_list == 1])
            var = sum((data[points_list == 0]-mean1)**2)/len(data[points_list == 0]),sum((data[points_list == 1]-mean2)**2)/len(data[points_list == 1])
        
    return random_kmeans, k_len, var

### Gaussian Mixture Model with Expectation Maximization - 1D GMM EM

In [52]:
# stars_kmean is the initialized k_means, can be random variables also
# data is your 1D matrix, only works with 1D matrix
# vark is the variance derived from k_means 
# proba_k is the probability derived from k_means how many points belonging to each cluster


def em(data, var_k, proba_k,stars_kmean):
    #initialize mean,len,var,proba from kmean
    counter = 1
    old_mean = stars_kmean
    new_mean = stars_kmean

    while counter < 100:
        #E-step, calculate posterior porbabilities with kmeans
        old_mean = new_mean
   
        g_matrix = (1/np.sqrt((2*np.pi*var_k)))*np.exp(-1*(data[:,np.newaxis]-old_mean)**2/(2*var_k))
        #print(g_matrix)
        
        den = (proba_k[0]*g_matrix[:,0])+(proba_k[1]*g_matrix[:,1])
        den = np.array([den,den]).T
        #print(den.shape)
        uij = (proba_k*g_matrix)/den
        #print(sum(uij))
        
        #M-step
        uj = sum(uij*(data[:,np.newaxis]))/sum(uij)
        #print(uj.shape)
        pj = sum(uij)/2700
        #print(pj.shape)
        varj = sum((uij*(data[:,np.newaxis]-uj)**2))/sum(uij)
        #print(varj.shape)
        
        #update parameters
        var_k = varj
        proba_k = pj
        new_mean = uj
        counter = counter+1
    
    return new_mean, proba_k, var_k


### Iterate and gather random consecutive blocks

In [53]:
def random_consecutive(iteration):
    random = 90
    y = []
    counter=0
    for i in range(iteration):
        xr_block= xr[random:random+8,:]
        xg_block= xg[random:random+8,:]
        xb_block= xb[random:random+8,:]
        
        y.append(xr_block)
        y.append(xg_block)
        y.append(xb_block)
          
        random = random+25
        counter = counter + 1

    return y

### Principal component analysis with whitening - PCA (whiten)

In [54]:
def pca_whiten(data):
    zero_mean = data-data.mean()
    
    cov = zero_mean.T/len(zero_mean[0])
    cov_matrix = zero_mean.dot(cov)
    #print(cov_matrix.shape)
    
    v, w = np.linalg.eigh(cov_matrix)
    
    whiten = w.T/v**0.5
    
    return whiten, v, w


### Independent component analysis - ICA

In [55]:
# z is your reduced dimension data matrix 


def ica(Z, learn_rate):
    N = 76800
    W = np.identity(4)   #weight matrix
    Y = W.dot(Z)  #estimated source
    I = np.identity(4)
    means = []
    counter = 1
    
    while counter < 10000 :
        
        #print(W)
        g = np.tanh(Y)
        f = (np.power(Y,3))
        W_delta = (N*(I)-g.dot(f.T))*(W)      
        #print(W_delta)
        
        
        W_new = W+(learn_rate*(W_delta))
        #print(W_new)
        Y_new = W_new.dot(Z)
        
        means.append((np.mean(W_new-W)))
        
        if np.allclose(W_new,W, rtol=1e-05):
            break
        
        W = W_new
        #print(W)
        
        Y = Y_new
        #print(Y)
        counter = counter + 1
        
        
        
    return W,Y, means, counter

### Signal to noise ratio - SNR

In [56]:
#original is your original signal
#recovered is your recovered signal
#length is the original data signal length

def snr_db(original,recovered,length):
        num = sum(original**2)
        #print(num)
        den = np.sum((original[:length]-recovered)**2)
        #print(den)
        return 10*np.log10(num/den)
    
    
    

### Non-negative matrix factorization -NMF

In [57]:
#data is your non-negative (amplitude only) data matrix
# vectors is how many vectors you want to keep (this is the reduced dimension)

def nmf(data, vectors):
    
    X = data
    x_len,y_len =data.shape
    W = np.random.random((x_len, vectors))  #my w is 513x30
    H = np.random.random((vectors, y_len))  #h.dot(w) = X (513,989), h need to be 30 x 989
    lh = (X-W.dot(H)).T
    rh = (X-W.dot(H))
    obj = 0.5*np.trace(lh.dot(rh))
    counter = 0
    
    while counter < 500:
        
        gamma = W/(W.dot(H).dot(H.T))
        gamma_rh = (W.dot(H).dot(H.T))-(X.dot(H.T))
        
        W = W - np.multiply(gamma,gamma_rh)
        #print(W)
       
        
        #update rules
        
        W_rh = (X.dot(H.T))/(W.dot(H).dot(H.T))
        W_new= np.multiply(W,W_rh)
        W = W_new
        
        H_rh = (W.T.dot(X))/(W.T.dot(W).dot(H))
        new_H = np.multiply(H,H_rh) 
        H = new_H
        
        counter = counter+1
        #print(counter)
        
        lh = (X-W.dot(H)).T
        rh = (X-W.dot(H))
        obj_new = 0.5*np.trace(lh.dot(rh))
        
        if np.allclose(obj_new, obj, rtol=1e-04):      #objective break
            break
            
        obj = obj_new
        
        
    return W, H

### NMF frequency extraction

In [58]:
def extract_mu_stft(data):
    x_list= []
    a1,a2,a3 = data.shape
    counter= 1
    for i in range(a3):
        x = librosa.stft(data[:,0,i], n_fft = 64, hop_length=48, window='blackman')
        y = librosa.stft(data[:,1,i], n_fft = 64, hop_length=48, window='blackman')
        z = librosa.stft(data[:,2,i], n_fft = 64, hop_length=48, window='blackman')
        
        new_x = x[2:7,:]
        #print(new_x.shape)
        new_y = y[2:7,:]
        new_z = z[2:7,:]
        
        cc = np.concatenate((new_x,new_y,new_z),axis=None)
        #print(cc.shape)
        
        x_list.append(cc)
       
        counter = counter+1
        
    return x_list

### K nearest neighbors - KNN

In [59]:
def KNN(x_train, x_test, y_train):
    x,y=x_train.shape
    y_train = y_train.tolist()
    min_val = []
    distance = []
    counter = 0
    for i_1 in range(28):
        for i in range(112):
                dist = np.linalg.norm(x_test[:,i_1] - x_train[:,i])
                distance.append(dist)
                counter = counter+1
                #print(distance)
                holder = distance.index(min(distance))
                #print(holder)
        min_val.append(holder)
        #print(min_val)
        distance = []
    y_pred = [y_train[val] for val in min_val] 
    
    return y_pred

### KNN accuracy

In [60]:
def accuracy(y_test,y_pred):
    tracker = []
    for i in range(len(y_test)):
        if y_test[i] == y_pred[i]:
            tracker.append(1)
        else:
            tracker.append(0)
            
        #print(tracker)
    return sum(tracker)